# Grupo 6
## Integrantes 
- Pablo Sarzosa
- Elisa Herrera
- Doménica Rodríguez 
- Andrés Albarracín

# 7.7. Redes densamente conectadas (DenseNet)

ResNet cambió significativamente la visión de cómo parametrizar las funciones en redes profundas. DenseNet (red convolucional densa) es, hasta cierto punto, la extensión lógica de esto [Huang et al., 2017]. Para entender cómo llegar a él, tomemos un pequeño desvío hacia las matemáticas.
## 7.7.1. De ResNet a DenseNet
Recuerde la expansión de Taylor para funciones. Para el punto x=0 se puede escribir como

   f(x)=f(0)+f′(0)x+f′′(0)2!x2+f′′′(0)3!x3+….
   
El punto clave es que descompone una función en términos de orden cada vez mayor. De manera similar, ResNet descompone funciones en

f(x)=x+g(x).

Es decir, ResNet descompone f en un término lineal simple y uno no lineal más complejo. ¿Qué pasa si queremos capturar (no necesariamente agregar) información más allá de dos términos? Una solución fue DenseNet [Huang et al., 2017](https://fcc.epn.edu.ec/d2l.ai/chapter_references/zreferences.html#huang-liu-van-der-maaten-ea-2017).

<div align="center">
    <img src="https://fcc.epn.edu.ec/d2l.ai/_images/densenet-block.svg">
    <br>
    Fig. 7.7.1 La principal diferencia entre ResNet (izquierda) y DenseNet (derecha) en conexiones entre capas: uso de suma y uso de concatenación.

</div>

Como se muestra en la Fig. 7.7.1, la diferencia clave entre ResNet y DenseNet es que, en el último caso, las salidas se concatenan (indicadas por [,] ) en lugar de sumarse. Como resultado, realizamos un mapeo de x a sus valores después de aplicar una secuencia de funciones cada vez más compleja:

x→[x,f1(x),f2([x,f1(x)]),f3([x,f1(x),f2([x,f1(x)])]),…].

Al final, todas estas funciones se combinan en MLP para reducir nuevamente la cantidad de características. En términos de implementación, esto es bastante simple: en lugar de agregar términos, los concatenamos. El nombre DenseNet surge del hecho de que el gráfico de dependencia entre variables se vuelve bastante denso. La última capa de tal cadena está densamente conectada a todas las capas anteriores. Las conexiones densas se muestran en la Fig. 7.7.2.

<div align="center">
    <img src="https://fcc.epn.edu.ec/d2l.ai/_images/densenet.svg">
    <br>
    Fig. 7.7.2 Conexiones densas en DenseNet.
</div>

## 7.7.2. Bloques densos
DenseNet utiliza la estructura modificada de "normalización, activación y convolución por lotes" de ResNet (consulte el ejercicio en la Sección 7.6). Primero, implementamos esta estructura de bloque de convolución.

In [ ]:
use strict;
use warnings;
use Data::Dump qw(dump);
use Data::Dumper qw(Dumper);
use AI::MXNet qw(mx);
use d2l;
use d2l::Animator;
use d2l::Accumulator;
use d2l::Timer;
IPerl->load_plugin('Chart::Plotly');

In [2]:
no strict 'subs';
sub conv_block_v2 {
my $num_channels = shift;
my $blk = NeuralNetworks::Sequential->new();
$blk->add(
NeuralNetworks::BatchNorm->new(),
NeuralNetworks::Activation->new('relu'),
NeuralNetworks::Conv2D->new(num_channels, kernel_size => 3, padding => 1)
);
return $blk;
}

Un bloque denso consta de varios bloques de convolución, cada uno de los cuales utiliza el mismo número de canales de salida. En la propagación directa, sin embargo, concatenamos la entrada y la salida de cada bloque de convolución en la dimensión del canal.

In [3]:
no strict 'subs';

sub conv_block_1 {
  my ($num_channels) = @_;
  my $blk = [];
  push @{$blk}, 'BatchNorm', 'Activation'->new('relu'), 'Conv2D'->new($num_channels, kernel_size=>3, padding=>1);
  return $blk;
}

package DenseBlock;
sub new_1 {
  my ($class, $num_convs, $num_channels) = @_;
  my $self = bless {}, $class;
  $self->{net} = [];
  for (0 .. ($num_convs-1)) {
    push @{$self->{net}}, conv_block_1($num_channels);
  }
  return $self;
}

sub forward_1 {
  my ($self, $X) = @_;
  for my $blk (@{$self->{net}}) {
    my $Y = $blk->($X);
    $X = [@{$X}, @{$Y}];
  }
  return $X;
}
1;

1

En el siguiente ejemplo, definimos una instancia de DenseBlock con 2 bloques de convolución de 10 canales de salida. Al usar una entrada con 3 canales, obtendremos una salida con 3+2×10=23 canales El número de canales de bloque de convolución controla el crecimiento del número de canales de salida en relación con el número de canales de entrada. Esto también se conoce como la tasa de crecimiento.

In [10]:
package DenseBlock;
use Moose;
use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use base qw(AI::MXNet::Gluon::Block);


my $blk = DenseBlock->new(num_convs => 2, num_channels => 10);
$blk->initialize(ctx => AI::MXNet->cpu());
my $X = AI::MXNet::NDArray->random(shape => [4, 3, 8, 8]);
my $Y = $blk->($X);
print $Y->shape, "\n";

Warning: Subroutine _build_conv_block redefined at reply input line 13.

Subroutine forward redefined at reply input line 22.

Subroutine conv_block redefined at reply input line 31.


Error: Odd number of paired arguments for method initialize at /usr/local/lib/perl5/site_perl/5.32.1/AI/MXNet/Gluon/Block.pm line 649.
	AI::MXNet::Gluon::Block::initialize(DenseBlock=HASH(0xad19200), "ctx", AI::MXNet::Context=HASH(0xb2dae80)) called at reply input line 43
	Eval::Closure::Sandbox_1292::__ANON__() called at /usr/local/lib/perl5/site_perl/5.32.1/Reply/Plugin/Defaults.pm line 71
	Reply::Plugin::Defaults::execute(Reply::Plugin::Defaults=HASH(0x48de110), CODE(0xb0fb6a0), CODE(0xb33ba10)) called at /usr/local/lib/perl5/site_perl/5.32.1/Reply.pm line 217
	Reply::_wrapped_plugin(Reply=HASH(0x48a5f98), ARRAY(0xb338c70), "execute", CODE(0xb33ba10)) called at /usr/local/lib/perl5/site_perl/5.32.1/Reply.pm line 215
	Reply::__ANON__(CODE(0xb33ba10)) called at /usr/local/lib/perl5/site_perl/5.32.1/Reply/Plugin/IPerl.pm line 28
	Reply::Plugin::IPerl::__ANON__() called at /usr/local/lib/perl5/site_perl/5.32.1/Capture/Tiny.pm line 382
	eval {...} called at /usr/local/lib/perl5/site_perl

## 7.7.3. Capas de transición
Dado que cada bloque denso aumentará el número de canales, agregar demasiados conducirá a un modelo excesivamente complejo. Se utiliza una capa de transición para controlar la complejidad del modelo. Reduce la cantidad de canales mediante el uso de la capa convolucional 1×1 y reduce a la mitad la altura y el ancho de la capa de agrupación promedio con un paso de 2, lo que reduce aún más la complejidad del modelo.

In [8]:
my $transition_block = sub {
my $num_channels = shift;
my $blk = mx->gluon->nn->Sequential();
$blk->add(mx->gluon->nn->BatchNorm(), mx->gluon->nn->Activation('relu'),
mx->gluon->nn->Conv2D($num_channels, kernel_size=>1),
mx->gluon->nn->AvgPool2D(pool_size=>2, strides=>2));
return $blk;
};

CODE(0xad20d10)

Aplique una capa de transición con 10 canales a la salida del bloque denso en el ejemplo anterior. Esto reduce el número de canales de salida a 10 y reduce a la mitad el alto y el ancho.

In [9]:
my $blk = mx->gluon->nn->Sequential();
$blk->add(mx->gluon->nn->BatchNorm(), mx->gluon->nn->Activation('relu'),
mx->gluon->nn->Conv2D(10, kernel_size => 1),
mx->gluon->nn->AvgPool2D(pool_size => 2, strides => 2));

my $input = mx->nd->random->uniform(shape => [4, 10, 4, 4]);
my $output = $blk->($input);

print $output->shape . "\n";

Error: Parameter 'batchnorm0_beta' has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks at /usr/local/lib/perl5/site_perl/5.32.1/AI/MXNet/Gluon/Parameter.pm line 252.
	AI::MXNet::Gluon::Parameter::_check_and_get(AI::MXNet::Gluon::Parameter=HASH(0xb327a20), undef, AI::MXNet::Context=HASH(0xb2f3710)) called at /usr/local/lib/perl5/site_perl/5.32.1/AI/MXNet/Gluon/Parameter.pm line 668
	AI::MXNet::Gluon::Parameter::data(AI::MXNet::Gluon::Parameter=HASH(0xb327a20), AI::MXNet::Context=HASH(0xb2f3710)) called at /usr/local/lib/perl5/site_perl/5.32.1/AI/MXNet/Gluon/Block.pm line 1181
	eval {...} called at /usr/local/lib/perl5/site_perl/5.32.1/AI/MXNet/Gluon/Block.pm line 1180
	AI::MXNet::Gluon::HybridBlock::forward(AI::MXNet::Gluon::NN::BatchNorm=HASH(0xb2ddbd0), AI::MXNet::NDArray=HASH(0x4f71ed8)) called at /usr/local/lib/perl5/site_perl/5

## 7.7.4. Modelo DenseNet
A continuación, construiremos un modelo DenseNet. DenseNet primero usa la misma capa convolucional única y la capa de agrupación máxima que en ResNet.

In [8]:
my $net = mx->gluon->nn->Sequential();
$net->add(mx->gluon->nn->Conv2D(64, kernel_size=>7, strides=>2, padding=>3));
$net->add(mx->gluon->nn->BatchNorm());
$net->add(mx->gluon->nn->Activation('relu'));
$net->add(mx->gluon->nn->MaxPool2D(pool_size=>3, strides=>2, padding=>1));

Error: Can't locate object method "gluon" via package "mx" (perhaps you forgot to load "mx"?) at reply input line 1.


Luego, similar a los cuatro módulos compuestos por bloques residuales que usa ResNet, DenseNet usa cuatro bloques densos. Similar a ResNet, podemos establecer la cantidad de capas convolucionales utilizadas en cada bloque denso. Aquí, lo configuramos en 4, de acuerdo con el modelo ResNet-18 en la Sección 7.6. Además, establecemos el número de canales (es decir, la tasa de crecimiento) para las capas convolucionales en el bloque denso en 32, por lo que se agregarán 128 canales a cada bloque denso.

En ResNet, la altura y el ancho se reducen entre cada módulo por un bloque residual con un paso de 2. Aquí, usamos la capa de transición para reducir a la mitad la altura y el ancho y reducir a la mitad el número de canales.

In [9]:
my $num_channels = 64;
my $growth_rate = 32;
my @num_convs_in_dense_blocks = (4, 4, 4, 4);

for (my $i = 0; $i < scalar @num_convs_in_dense_blocks; $i++) {
my $num_convs = $num_convs_in_dense_blocks[$i];
$net->add(DenseBlock->new(num_convs => $num_convs, growth_rate => $growth_rate));
$num_channels += $num_convs * $growth_rate;
if ($i != scalar(@num_convs_in_dense_blocks) - 1) {
$num_channels = int($num_channels / 2);
$net->add(transition_block->new(num_channels => $num_channels));
}
}

Error: Use of inherited AUTOLOAD for non-method DenseBlock::DenseBlock() is no longer allowed at reply input line 7.


De manera similar a ResNet, una capa de agrupación global y una capa totalmente conectada se conectan al final para producir la salida.

In [13]:


my @operations = (
'BATCH_NORM',
'ACTIVATION_RELU',
'GLOBAL_AVG_POOL_2D',
sub {
my ($input, $num_classes) = @_;
return dense($input, $num_classes);
}
);

Error: Can't call method "add" on unblessed reference at reply input line 7.


## 7.7.5. Entrenamiento
Dado que aquí estamos usando una red más profunda, en esta sección reduciremos la altura y el ancho de entrada de 224 a 96 para simplificar el cálculo.

In [12]:
use AI::FANN;
# Define la arquitectura de la red neuronal usando AI::FRANN
my $net = AI::NNFlex->new(
  layers => [
    input_layer(1, 96, 96),
    convolutional_layer(64, 7, 2, 3, 'relu'),
    max_pooling_layer(3, 2, 1),
    batch_normalization_layer(),
    activation_layer('relu'),
    global_average_pooling_layer(),
    dense_layer(10, 'linear'),
  ],
  loss => 'softmax_cross_entropy',
  learning_rate => 0.1,
  batch_size => 256,
  epochs => 10,
);



Error: Can't locate D2L/AI/MXNet.pm in @INC (you may need to install the D2L::AI::MXNet module) (@INC contains: /usr/local/bin/../lib /home/jupyter/perl5/lib/perl5/5.32.1/x86_64-linux /home/jupyter/perl5/lib/perl5/5.32.1 /home/jupyter/perl5/lib/perl5/x86_64-linux /home/jupyter/perl5/lib/perl5 /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux /usr/local/lib/perl5/site_perl/5.32.1 /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux /usr/local/lib/perl5/site_perl/5.32.1 /usr/local/lib/perl5/5.32.1/x86_64-linux /usr/local/lib/perl5/5.32.1) at reply input line 4.
BEGIN failed--compilation aborted at reply input line 4.



In [ ]:
# Cargar los datos de entrenamiento y prueba
my ($train_iter, $test_iter) = load_data_fashion_mnist(batch_size, 96);

# Entrenar la red neuronal
$net->train($train_iter, $test_iter);

In [ ]:
my ($model_file_name, $is_train, $animator) = ('GoogLeNet.mdl', 1);

if ($is_train){

  $animator = d2l->train_ch6($net, $train_iter, $test_iter, $num_epochs, $lr, d2l->try_gpu());

  $net->save_parameters($model_file_name);

  $animator->plot;

}else{

  $net->load_parameters($model_file_name);

}

## 7.7.6. Resumen
- En términos de conexiones entre capas, a diferencia de ResNet, donde las entradas y salidas se suman, DenseNet concatena entradas y salidas en la dimensión del canal.

- Los componentes principales que componen DenseNet son bloques densos y capas de transición.

- Necesitamos mantener la dimensionalidad bajo control al componer la red agregando capas de transición que reducen nuevamente la cantidad de canales.
## 7.7.7. Ejercicios
1. ¿Por qué usamos la agrupación promedio en lugar de la agrupación máxima en la capa de transición?

2. Una de las ventajas mencionadas en el documento de DenseNet es que los parámetros de su modelo son más pequeños que los de ResNet. ¿Por qué es este el caso?

3. Un problema por el que se ha criticado a DenseNet es su alto consumo de memoria.

   1. ¿Es este realmente el caso? Intenta cambiar la forma de entrada a 224 × 224
   para ver el consumo real de memoria de la GPU.

   2. ¿Puede pensar en un medio alternativo para reducir el consumo de memoria? ¿Cómo necesitarías cambiar el marco?

4. Implemente las diversas versiones de DenseNet presentadas en la Tabla 1 del documento de DenseNet [Huang et al., 2017].

5. Diseñe un modelo basado en MLP aplicando la idea de DenseNet. Aplíquelo a la tarea de predicción del precio de la vivienda en la Sección 4.10.